In [4]:
import torch, tiktoken

C:\Users\radcl\AppData\Roaming\Python\Python313\site-packages\torch\_subclasses\functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [5]:
with open ("thelostrace.txt", "r") as f:
    raw_text = f.read()


In [6]:
tokenizer = tiktoken.get_encoding("gpt2")

In [7]:
enc_text = tokenizer.encode(raw_text)
print(enc_text) 

[10606, 273, 1229, 27846, 546, 683, 290, 19338, 2945, 465, 8761, 13, 679, 373, 645, 26769, 11, 475, 339, 750, 407, 588, 262, 1295, 13, 22676, 7150, 8278, 477, 546, 11, 511, 424, 297, 268, 13737, 25136, 503, 262, 19606, 13, 383, 5391, 8025, 2957, 287, 290, 503, 1871, 606, 11, 3360, 1341, 35355, 262, 5743, 286, 257, 24343, 500, 11, 810, 2744, 273, 1229, 714, 17841, 866, 379, 262, 5509, 12, 35011, 11061, 13, 40659, 11, 832, 257, 36788, 287, 262, 8222, 11, 339, 714, 766, 1497, 284, 262, 11747, 13494, 18639, 326, 27388, 286, 262, 16069, 881, 18485, 284, 262, 7421, 11, 326, 547, 262, 12269, 286, 45294, 13, 198, 198, 818, 883, 12269, 262, 4097, 270, 4039, 11, 5481, 1229, 262, 44153, 11, 373, 4385, 284, 20605, 74, 11, 284, 15350, 2402, 884, 4970, 355, 1244, 1208, 326, 835, 13, 2744, 273, 1229, 14869, 465, 11762, 319, 465, 17514, 290, 2068, 2945, 465, 2239, 13, 2399, 36797, 373, 2233, 407, 691, 284, 262, 36292, 286, 262, 23716, 82, 11, 475, 635, 284, 262, 1109, 326, 339, 16555, 1752, 517, 284, 

In [8]:
vocab_size = 4
output_dim = 3
inputs = torch.nn.Embedding(vocab_size, output_dim)
print(inputs.weight)

Parameter containing:
tensor([[ 1.1802,  0.3441, -1.6928],
        [-0.3828, -2.0360, -1.1997],
        [-0.8329,  0.2844, -0.5540],
        [-0.0102, -0.2871,  0.0961]], requires_grad=True)


In [9]:
inputs = inputs.weight.data
print(inputs)


tensor([[ 1.1802,  0.3441, -1.6928],
        [-0.3828, -2.0360, -1.1997],
        [-0.8329,  0.2844, -0.5540],
        [-0.0102, -0.2871,  0.0961]])


In [10]:
inputs.shape

torch.Size([4, 3])

In [11]:
for row in inputs:
    print(row.tolist())

[1.1801810264587402, 0.3441344201564789, -1.692831039428711]
[-0.3827906548976898, -2.035987615585327, -1.1997425556182861]
[-0.8328617215156555, 0.28440597653388977, -0.5540288686752319]
[-0.010181297548115253, -0.2870625853538513, 0.09614706039428711]


In [12]:
x = torch.Tensor([1.1,2.3])
y = torch.Tensor([3.4,-2.1])

In [13]:
torch.dot(x,y) # dot = x1 *x2 +y1*y2

tensor(-1.0900)

In [14]:
query = inputs[2]
print(query)


tensor([-0.8329,  0.2844, -0.5540])


In [15]:
for i in range(len(inputs)):
    print(torch.dot(query,inputs[i]))

tensor(0.0528)
tensor(0.4045)
tensor(1.0815)
tensor(-0.1264)


In [16]:
attention_scores2 = torch.empty(len(inputs))
for i in range(len(inputs)):
    attention_scores2[i] = torch.dot(query,inputs[i])
print(attention_scores2)

tensor([ 0.0528,  0.4045,  1.0815, -0.1264])


In [17]:
def softmax(x):
    torch.exp(x)/ torch.exp(x)

In [18]:
torch.softmax(attention_scores2, dim = 0)

tensor([0.1652, 0.2348, 0.4620, 0.1381])

In [19]:
attention_scores2.sum()


tensor(1.4123)

In [20]:
context_vector_2 = torch.zeros (query.shape)
for i in range(len(attention_scores2)):
    context_vector_2 += attention_scores2[i] * inputs[i]
context_vector_2

tensor([-0.9919, -0.4614, -1.1860])

In [21]:
attention_weights = inputs @ inputs.T
attention_weights

tensor([[ 4.3769,  0.8785,  0.0528, -0.2736],
        [ 0.8785,  5.7312,  0.4045,  0.4730],
        [ 0.0528,  0.4045,  1.0815, -0.1264],
        [-0.2736,  0.4730, -0.1264,  0.0918]])

In [22]:
attention_weights[0].sum


<function Tensor.sum>

In [23]:
context_vectors = attention_weights @ inputs
context_vectors

tensor([[  4.7881,  -0.1889,  -8.5190],
        [ -1.4987, -11.3870,  -8.5417],
        [ -0.9919,  -0.4614,  -1.1860],
        [ -0.3996,  -1.1195,  -0.0255]])